In [1]:
import pydicom
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import os
import csv
from datetime import datetime
from collections import defaultdict

DATA_FILE = r"C:\Users\maryam\Desktop\rad.csv"

THEMES = {
    "Light": {"bg": "#ffffff", "fg": "#333", "button": "#3498db", "card": "#f4f4f4", "border": "#ddd"},
    "Dark": {"bg": "#1e1e1e", "fg": "#ffffff", "button": "#00b7c2", "card": "#2c2c2c", "border": "#444"},
}

current_theme = "Light"

# CSV file init
def init_csv_file():
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['PatientName', 'StudyDate', 'CTDIvol'])

# Save to CSV
patient_records = set()
def save_to_csv(patient_name, study_date, ctdi):
    key = (patient_name, study_date, ctdi)
    if key not in patient_records:
        with open(DATA_FILE, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([patient_name, study_date, ctdi])
            patient_records.add(key)

# Load CSV
def load_csv_data():
    data = []
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                data.append({
                    'PatientName': row['PatientName'],
                    'StudyDate': row['StudyDate'],
                    'CTDIvol': float(row['CTDIvol'])
                })
    return data

def parse_date(dicom_date):
    return datetime.strptime(dicom_date, "%Y%m%d")

def load_dicom_info(filepath):
    ds = pydicom.dcmread(filepath)
    patient_name = str(ds.PatientName)
    study_date = str(ds.StudyDate)
    ctdi = None
    try:
        if (0x0018, 0x9345) in ds:
            ctdi = float(ds[(0x0018, 0x9345)].value)
    except Exception as e:
        print(f"Error reading CTDIvol: {e}")
    return ds, patient_name, study_date, ctdi

def apply_theme():
    theme = THEMES[current_theme]
    root.configure(bg=theme['bg'])
    top_frame.configure(bg=theme['bg'])
    control_frame.configure(bg=theme['bg'])
    result_label.configure(bg=theme['bg'], fg=theme['fg'])
    footer.configure(bg=theme['bg'], fg=theme['fg'])

def toggle_theme():
    global current_theme
    current_theme = "Dark" if current_theme == "Light" else "Light"
    apply_theme()

def clear_all():
    for widget in image_canvas_frame.winfo_children():
        widget.destroy()
    result_label.config(text="")

def select_files():
    filepaths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not filepaths:
        return

    clear_all()
    patient_data = defaultdict(list)
    name_set = set()

    for filepath in filepaths:
        ds, name, date, ctdi = load_dicom_info(filepath)
        if ctdi is None:
            continue

        key = (name, date, ctdi)
        if key in patient_records:
            continue

        name_set.add(name)
        save_to_csv(name, date, ctdi)
        patient_data[name].append((parse_date(date), ctdi, filepath, date))

    if len(name_set) > 1:
        messagebox.showwarning("تحذير", "الصور ليست لنفس المريض! الرجاء التأكد من اختيار صور لمريض واحد فقط.")

    image_index = 0
    for name, records in patient_data.items():
        records.sort()
        for record in records:
            date_obj, ctdi, filepath, date_str = record
            ds = pydicom.dcmread(filepath)
            img_array = ds.pixel_array
            img_pil = Image.fromarray(img_array)
            img_tk = ImageTk.PhotoImage(img_pil)

            frame = tk.Frame(image_canvas_frame, bg=THEMES[current_theme]['card'], bd=0,
                             highlightbackground=THEMES[current_theme]['border'], highlightthickness=1)
            frame.pack(fill='x', padx=20, pady=10)

            image_label = tk.Label(frame, image=img_tk, bg=THEMES[current_theme]['card'])
            image_label.image = img_tk
            image_label.pack(side="left", padx=10)

            info_text = f"Name: {name}\nDate: {date_str[:4]}-{date_str[4:6]}-{date_str[6:]}\nCTDIvol: {ctdi:.2f}"
            info_label = tk.Label(frame, text=info_text, font=("Segoe UI", 11), justify="left",
                                  bg=THEMES[current_theme]['card'], fg="green")
            info_label.pack(side="left", padx=10)

            image_index += 1

    result_label.config(text="")
    all_data = load_csv_data()
    grouped_by_patient = defaultdict(list)

    for entry in all_data:
        grouped_by_patient[entry['PatientName']].append((parse_date(entry['StudyDate']), entry['CTDIvol']))

    summary_lines = []
    for name, entries in grouped_by_patient.items():
        entries.sort()
        if not entries:
            continue
        first_date = entries[0][0]
        one_year_later = first_date.replace(year=first_date.year + 1)

        yearly_total = sum(ctdi for date, ctdi in entries if first_date <= date <= one_year_later)
        overall_total = sum(ctdi for _, ctdi in entries)

        date_range_str = f"{first_date.strftime('%Y-%m-%d')} → {one_year_later.strftime('%Y-%m-%d')}"
        summary_lines.append(f"{name} → {yearly_total:.2f} mGy from {date_range_str}")
        summary_lines.append(f"{name} → مجموع الجرعة الكلي: {overall_total:.2f} mGy")

    # عرض النتائج بتنسيق خاص
    result_label.config(text="\n\n".join(summary_lines), font=("Segoe UI", 12, "bold"), fg="green")

# GUI setup
root = tk.Tk()
root.title("DICOM CTDIvol Analyzer")
root.geometry("1200x800")
root.configure(bg=THEMES[current_theme]['bg'])

init_csv_file()

# Frames
top_frame = tk.Frame(root)
top_frame.pack(pady=10)

control_frame = tk.Frame(root)
control_frame.pack(pady=5)

canvas_frame = tk.Frame(root)
canvas_frame.pack(fill=tk.BOTH, expand=True, padx=20)

scrollbar = tk.Scrollbar(canvas_frame, orient=tk.VERTICAL)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

image_canvas = tk.Canvas(canvas_frame, yscrollcommand=scrollbar.set, bd=0, highlightthickness=0)
image_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
scrollbar.config(command=image_canvas.yview)

image_canvas_frame = tk.Frame(image_canvas)
image_canvas.create_window((0, 0), window=image_canvas_frame, anchor='nw')

def on_frame_configure(event):
    image_canvas.configure(scrollregion=image_canvas.bbox("all"))

image_canvas_frame.bind("<Configure>", on_frame_configure)

# Controls
title = tk.Label(top_frame, text="CTDIvol Viewer & Yearly Dose Calculator", font=("Segoe UI", 18, "bold"),
                 fg=THEMES[current_theme]['fg'], bg=THEMES[current_theme]['bg'])
title.pack()

style_btn = {
    'font': ("Segoe UI", 10, "bold"),
    'bd': 0,
    'fg': "white",
    'activeforeground': "white",
    'relief': tk.FLAT,
    'width': 18,
    'pady': 10,
    'cursor': "hand2"
}

select_btn = tk.Button(control_frame, text="Select DICOM Files", command=select_files,
                       bg=THEMES[current_theme]['button'], **style_btn)
select_btn.grid(row=0, column=0, padx=8)

reset_btn = tk.Button(control_frame, text="Reset for New Patient", command=clear_all,
                      bg="#e67e22", **style_btn)
reset_btn.grid(row=0, column=1, padx=8)

theme_btn = tk.Button(control_frame, text="Toggle Theme", command=toggle_theme,
                      bg="#8e44ad", **style_btn)
theme_btn.grid(row=0, column=2, padx=8)

result_label = tk.Label(root, text="", font=("Segoe UI", 12), fg="#2980b9", bg=THEMES[current_theme]['bg'])
result_label.pack(pady=10)

footer = tk.Label(root, text="Developed with ❤️ using pydicom", font=("Segoe UI", 9), fg="#aaa",
                  bg=THEMES[current_theme]['bg'])
footer.pack(side="bottom", pady=10)

apply_theme()
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from PIL import Image, ImageTk
import pydicom
import numpy as np
import os
import csv
from datetime import datetime

# إعداد الملف CSV
csv_file = "rad.csv"
fieldnames = ["PatientName", "StudyDate", "CTDIvol"]

if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

# إنشاء النافذة
root = tk.Tk()
root.title("DICOM Dose Tracker")
root.geometry("1400x800")
root.configure(bg="#f0f8ff")

# خلفية ديكورية لطيفة
bg_image = Image.open(r"background-medical.jpg")  # ضع صورة خلفية لطيفة هنا بنفس المجلد
bg_image = bg_image.resize((1400, 800))
bg_photo = ImageTk.PhotoImage(bg_image)

bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

frame = tk.Frame(root, bg="#ffffff", bd=3, relief="ridge")
frame.pack(padx=20, pady=20, fill="both", expand=True)

canvas = tk.Canvas(frame, bg="#ffffff")
scroll_y = tk.Scrollbar(frame, orient="vertical", command=canvas.yview)
scroll_frame = tk.Frame(canvas, bg="#ffffff")

canvas.create_window((0, 0), window=scroll_frame, anchor="nw")
canvas.configure(yscrollcommand=scroll_y.set)

canvas.pack(side="left", fill="both", expand=True)
scroll_y.pack(side="right", fill="y")

image_labels = {}

def extract_ctdi(dicom_data):
    try:
        return float(dicom_data[0x0018, 0x9345].value)  # CTDIvol
    except:
        return 0.0

def extract_date(ds):
    try:
        return datetime.strptime(ds.StudyDate, "%Y%m%d").date()
    except:
        return "Unknown"

def process_dicom_files():
    filenames = filedialog.askopenfilenames(filetypes=[("DICOM Files", "*.dcm")])
    patient_doses = {}

    for widget in scroll_frame.winfo_children():
        widget.destroy()

    for filename in filenames:
        ds = pydicom.dcmread(filename)
        ctdi = extract_ctdi(ds)
        patient = str(ds.PatientName)
        study_date = extract_date(ds)

        if patient not in patient_doses:
            patient_doses[patient] = []
        patient_doses[patient].append((study_date, ctdi))

        # إضافة البيانات إلى CSV
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow({
                "PatientName": patient,
                "StudyDate": study_date,
                "CTDIvol": ctdi
            })

        # عرض الصورة
        if "PixelData" in ds:
            pixel_array = ds.pixel_array
            if np.max(pixel_array) > 255:
                pixel_array = (pixel_array / np.max(pixel_array)) * 255
            img = Image.fromarray(pixel_array.astype(np.uint8))
            img = img.resize((256, 256))
            img_tk = ImageTk.PhotoImage(img)

            panel = tk.Label(scroll_frame, image=img_tk)
            panel.image = img_tk
            panel.pack(pady=10)

            # كتابة التفاصيل
            tk.Label(scroll_frame, text=f"اسم المريض: {patient}", font=("Arial", 12), bg="#ffffff").pack()
            tk.Label(scroll_frame, text=f"تاريخ الفحص: {study_date}", font=("Arial", 12), bg="#ffffff").pack()
            tk.Label(scroll_frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 12, "bold"), fg="blue", bg="#ffffff").pack()
            tk.Label(scroll_frame, text="-"*50, bg="#ffffff").pack()

    # حساب الجرعة الكلية لكل مريض
    tk.Label(scroll_frame, text="الجرعة الكلية لكل حالة:", font=("Arial", 14, "bold"), fg="green", bg="#ffffff").pack(pady=20)
    for patient, records in patient_doses.items():
        total_dose = sum([r[1] for r in records])
        tk.Label(scroll_frame, text=f"{patient}: {total_dose:.2f} mGy", font=("Arial", 12), bg="#ffffff").pack()

    scroll_frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))

# زر التحميل
upload_btn = tk.Button(root, text="تحميل ملفات DICOM", command=process_dicom_files, font=("Arial", 14), bg="#4CAF50", fg="white", padx=10, pady=5)
upload_btn.pack(pady=10)

root.mainloop()


FileNotFoundError: [Errno 2] No such file or directory: 'background_image.jpg'

In [2]:
pip install pydicom pillow tkinter pandas python-hl7


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maryam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from PIL import Image, ImageTk
import pydicom
import numpy as np
import os
import csv
from datetime import datetime

# إعداد الملف CSV
csv_file = "rad.csv"
fieldnames = ["PatientName", "StudyDate", "CTDIvol"]

if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

# إنشاء النافذة
root = tk.Tk()
root.title("DICOM Dose Tracker")
root.geometry("1400x800")
root.configure(bg="#f0f8ff")

# خلفية ديكورية لطيفة
bg_image = Image.open("background-medical.jpg")  # ضع صورة خلفية لطيفة هنا بنفس المجلد
bg_image = bg_image.resize((1400, 800))
bg_photo = ImageTk.PhotoImage(bg_image)

bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

frame = tk.Frame(root, bg="#ffffff", bd=3, relief="ridge")
frame.pack(padx=20, pady=20, fill="both", expand=True)

canvas = tk.Canvas(frame, bg="#ffffff")
scroll_y = tk.Scrollbar(frame, orient="vertical", command=canvas.yview)
scroll_frame = tk.Frame(canvas, bg="#ffffff")

canvas.create_window((0, 0), window=scroll_frame, anchor="nw")
canvas.configure(yscrollcommand=scroll_y.set)

canvas.pack(side="left", fill="both", expand=True)
scroll_y.pack(side="right", fill="y")

image_labels = {}

def extract_ctdi(dicom_data):
    try:
        return float(dicom_data[0x0018, 0x9345].value)  # CTDIvol
    except:
        return 0.0

def extract_date(ds):
    try:
        return datetime.strptime(ds.StudyDate, "%Y%m%d").date()
    except:
        return "Unknown"

def process_dicom_files():
    filenames = filedialog.askopenfilenames(filetypes=[("DICOM Files", "*.dcm")])
    patient_doses = {}

    for widget in scroll_frame.winfo_children():
        widget.destroy()

    for filename in filenames:
        ds = pydicom.dcmread(filename)
        ctdi = extract_ctdi(ds)
        patient = str(ds.PatientName)
        study_date = extract_date(ds)

        if patient not in patient_doses:
            patient_doses[patient] = []
        patient_doses[patient].append((study_date, ctdi))

        # إضافة البيانات إلى CSV
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow({
                "PatientName": patient,
                "StudyDate": study_date,
                "CTDIvol": ctdi
            })

        # عرض الصورة
        if "PixelData" in ds:
            pixel_array = ds.pixel_array
            if np.max(pixel_array) > 255:
                pixel_array = (pixel_array / np.max(pixel_array)) * 255
            img = Image.fromarray(pixel_array.astype(np.uint8))
            img = img.resize((256, 256))
            img_tk = ImageTk.PhotoImage(img)

            panel = tk.Label(scroll_frame, image=img_tk)
            panel.image = img_tk
            panel.pack(pady=10)

            # كتابة التفاصيل
            tk.Label(scroll_frame, text=f"اسم المريض: {patient}", font=("Arial", 12), bg="#ffffff").pack()
            tk.Label(scroll_frame, text=f"تاريخ الفحص: {study_date}", font=("Arial", 12), bg="#ffffff").pack()
            tk.Label(scroll_frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 12, "bold"), fg="blue", bg="#ffffff").pack()
            tk.Label(scroll_frame, text="-"*50, bg="#ffffff").pack()

    # حساب الجرعة الكلية لكل مريض
    tk.Label(scroll_frame, text="الجرعة الكلية لكل حالة:", font=("Arial", 14, "bold"), fg="green", bg="#ffffff").pack(pady=20)
    for patient, records in patient_doses.items():
        total_dose = sum([r[1] for r in records])
        tk.Label(scroll_frame, text=f"{patient}: {total_dose:.2f} mGy", font=("Arial", 12), bg="#ffffff").pack()

    scroll_frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))

# زر التحميل
upload_btn = tk.Button(root, text="تحميل ملفات DICOM", command=process_dicom_files, font=("Arial", 14), bg="#4CAF50", fg="white", padx=10, pady=5)
upload_btn.pack(pady=10)

root.mainloop()


TclError: image "pyimage2" doesn't exist

In [5]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
from datetime import datetime
import hl7

# Globals
data_list = []
hl7_messages = {}

def read_dicom_files():
    file_paths = filedialog.askopenfilenames(title="Select DICOM files", filetypes=[("DICOM files", "*.dcm")])
    for file_path in file_paths:
        try:
            ds = pydicom.dcmread(file_path)
            name = getattr(ds, "PatientName", "Unknown")
            date = getattr(ds, "StudyDate", "00000000")
            ctdi = 0
            for elem in ds.iterall():
                if "CTDIvol" in str(elem.name):
                    try:
                        ctdi = float(elem.value)
                    except:
                        pass
            img = get_image(ds)
            display_image(img, name, date, ctdi)
            save_data(name, date, ctdi)
        except Exception as e:
            print("Failed to load:", e)

def get_image(ds):
    try:
        arr = ds.pixel_array
        image = Image.fromarray(arr)
        return image.resize((256, 256))
    except:
        return Image.new("L", (256, 256), color=128)

def display_image(img, name, date, ctdi):
    frame = tk.Frame(images_frame, bg="#ffffff", bd=2, relief="groove")
    frame.pack(side="left", padx=10, pady=10)
    imgtk = ImageTk.PhotoImage(img.convert("RGB"))
    lbl_img = tk.Label(frame, image=imgtk)
    lbl_img.image = imgtk
    lbl_img.pack()
    tk.Label(frame, text=f"Name: {name}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"Date: {format_date(date)}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 10), bg="#ffffff", fg="green").pack()

def format_date(d):
    try:
        return datetime.strptime(d, "%Y%m%d").strftime("%d-%m-%Y")
    except:
        return "Unknown"

def save_data(name, date, ctdi):
    global data_list
    data_list.append({
        "PatientName": str(name),
        "StudyDate": date,
        "CTDIvol": ctdi
    })

def generate_csv_and_hl7():
    global data_list, hl7_messages
    df = pd.DataFrame(data_list)
    df['StudyDate'] = pd.to_datetime(df['StudyDate'], format="%Y%m%d", errors="coerce")
    df.dropna(inplace=True)

    output = []
    hl7_messages.clear()

    for patient in df['PatientName'].unique():
        patient_df = df[df['PatientName'] == patient].sort_values("StudyDate")
        start_date = patient_df['StudyDate'].min()
        end_date = start_date + pd.DateOffset(years=1)
        yearly_df = patient_df[(patient_df['StudyDate'] >= start_date) & (patient_df['StudyDate'] <= end_date)]
        total = yearly_df['CTDIvol'].sum()

        # Save patient HL7 message
        msg = hl7.parse(f"MSH|^~\\&|CT|HOSPITAL|RAD|HOSPITAL|{datetime.now().strftime('%Y%m%d')}||ORM^O01|MSG00001|P|2.3\rPID|||{patient}||\rOBX|1|NM|CTDIvol||{total}|mGy")
        hl7_messages[str(patient)] = msg

        output.append({
            "PatientName": patient,
            "StartDate": start_date.strftime("%Y-%m-%d"),
            "EndDate": end_date.strftime("%Y-%m-%d"),
            "Total_CTDIvol": total
        })

    result_df = pd.DataFrame(output)
    result_df.to_csv("rad.csv", index=False)
    print("Saved to rad.csv")

def search_hl7():
    name = search_entry.get().strip()
    if name in hl7_messages:
        text.delete(1.0, tk.END)
        text.insert(tk.END, str(hl7_messages[name]))
    else:
        text.delete(1.0, tk.END)
        text.insert(tk.END, "No HL7 message found for this patient.")

# GUI setup
root = tk.Tk()
root.title("Smart DICOM Viewer with CTDIvol")
root.geometry("1200x700")

bg_img = Image.open("background.jpg").resize((1200, 700))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.pack(pady=10)

images_frame = tk.Frame(root, bg="white")
images_frame.pack(fill="both", expand=True)

bottom_frame = tk.Frame(root, bg="#f0f0f0")
bottom_frame.pack(fill="x", pady=5)

tk.Button(bottom_frame, text="📄 Save CSV + HL7", command=generate_csv_and_hl7, bg="blue", fg="white").pack(side="left", padx=10)
search_entry = tk.Entry(bottom_frame)
search_entry.pack(side="left")
tk.Button(bottom_frame, text="🔍 Find HL7 by Patient", command=search_hl7).pack(side="left", padx=5)

text = tk.Text(bottom_frame, height=5)
text.pack(side="left", fill="x", expand=True, padx=10)

root.mainloop()


ModuleNotFoundError: No module named 'pandas'

In [6]:
pip install pandas


   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB 991.0 kB/s eta 0:00:12
   ---------------------------------------- 0.1/11.6 MB 1.3 MB/s eta 0:00:09
    --------------------------------------- 0.2/11.6 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.4/11.6 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.6/11.6 MB 2.7 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/11.6 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 0.9/11.6 MB 3.0 MB/s eta 0:00:04
   --- ------------------------------------ 1.1/11.6 MB 3.2 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.6 MB 3.2 MB/s eta 0:00:04
   ---- ----------------------------------- 1.4/11.6 MB 3.1 MB/s eta 0:00:04
   ----- ---------------------------------- 1.5/11.6 MB 3.2 MB/s eta 0:00:04
   ------ --------------------------------- 1.8/11.6 MB 3.4 MB/s eta 0:00:03
   -


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maryam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
from datetime import datetime
import hl7

# Globals
data_list = []
hl7_messages = {}

def read_dicom_files():
    file_paths = filedialog.askopenfilenames(title="Select DICOM files", filetypes=[("DICOM files", "*.dcm")])
    for file_path in file_paths:
        try:
            ds = pydicom.dcmread(file_path)
            name = getattr(ds, "PatientName", "Unknown")
            date = getattr(ds, "StudyDate", "00000000")
            ctdi = 0
            for elem in ds.iterall():
                if "CTDIvol" in str(elem.name):
                    try:
                        ctdi = float(elem.value)
                    except:
                        pass
            img = get_image(ds)
            display_image(img, name, date, ctdi)
            save_data(name, date, ctdi)
        except Exception as e:
            print("Failed to load:", e)

def get_image(ds):
    try:
        arr = ds.pixel_array
        image = Image.fromarray(arr)
        return image.resize((256, 256))
    except:
        return Image.new("L", (256, 256), color=128)

def display_image(img, name, date, ctdi):
    frame = tk.Frame(images_frame, bg="#ffffff", bd=2, relief="groove")
    frame.pack(side="left", padx=10, pady=10)
    imgtk = ImageTk.PhotoImage(img.convert("RGB"))
    lbl_img = tk.Label(frame, image=imgtk)
    lbl_img.image = imgtk
    lbl_img.pack()
    tk.Label(frame, text=f"Name: {name}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"Date: {format_date(date)}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 10), bg="#ffffff", fg="green").pack()

def format_date(d):
    try:
        return datetime.strptime(d, "%Y%m%d").strftime("%d-%m-%Y")
    except:
        return "Unknown"

def save_data(name, date, ctdi):
    global data_list
    data_list.append({
        "PatientName": str(name),
        "StudyDate": date,
        "CTDIvol": ctdi
    })

def generate_csv_and_hl7():
    global data_list, hl7_messages
    df = pd.DataFrame(data_list)
    df['StudyDate'] = pd.to_datetime(df['StudyDate'], format="%Y%m%d", errors="coerce")
    df.dropna(inplace=True)

    output = []
    hl7_messages.clear()

    for patient in df['PatientName'].unique():
        patient_df = df[df['PatientName'] == patient].sort_values("StudyDate")
        start_date = patient_df['StudyDate'].min()
        end_date = start_date + pd.DateOffset(years=1)
        yearly_df = patient_df[(patient_df['StudyDate'] >= start_date) & (patient_df['StudyDate'] <= end_date)]
        total = yearly_df['CTDIvol'].sum()

        # Save patient HL7 message
        msg = hl7.parse(f"MSH|^~\\&|CT|HOSPITAL|RAD|HOSPITAL|{datetime.now().strftime('%Y%m%d')}||ORM^O01|MSG00001|P|2.3\rPID|||{patient}||\rOBX|1|NM|CTDIvol||{total}|mGy")
        hl7_messages[str(patient)] = msg

        output.append({
            "PatientName": patient,
            "StartDate": start_date.strftime("%Y-%m-%d"),
            "EndDate": end_date.strftime("%Y-%m-%d"),
            "Total_CTDIvol": total
        })

    result_df = pd.DataFrame(output)
    result_df.to_csv("rad.csv", index=False)
    print("Saved to rad.csv")

def search_hl7():
    name = search_entry.get().strip()
    if name in hl7_messages:
        text.delete(1.0, tk.END)
        text.insert(tk.END, str(hl7_messages[name]))
    else:
        text.delete(1.0, tk.END)
        text.insert(tk.END, "No HL7 message found for this patient.")

# GUI setup
root = tk.Tk()
root.title("Smart DICOM Viewer with CTDIvol")
root.geometry("1200x700")

bg_img = Image.open("background.jpg").resize((1200, 700))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.pack(pady=10)

images_frame = tk.Frame(root, bg="white")
images_frame.pack(fill="both", expand=True)

bottom_frame = tk.Frame(root, bg="#f0f0f0")
bottom_frame.pack(fill="x", pady=5)

tk.Button(bottom_frame, text="📄 Save CSV + HL7", command=generate_csv_and_hl7, bg="blue", fg="white").pack(side="left", padx=10)
search_entry = tk.Entry(bottom_frame)
search_entry.pack(side="left")
tk.Button(bottom_frame, text="🔍 Find HL7 by Patient", command=search_hl7).pack(side="left", padx=5)

text = tk.Text(bottom_frame, height=5)
text.pack(side="left", fill="x", expand=True, padx=10)

root.mainloop()


ModuleNotFoundError: No module named 'hl7'

In [8]:
pip install hl7


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maryam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
from datetime import datetime
import hl7

# Globals
data_list = []
hl7_messages = {}

def read_dicom_files():
    file_paths = filedialog.askopenfilenames(title="Select DICOM files", filetypes=[("DICOM files", "*.dcm")])
    for file_path in file_paths:
        try:
            ds = pydicom.dcmread(file_path)
            name = getattr(ds, "PatientName", "Unknown")
            date = getattr(ds, "StudyDate", "00000000")
            ctdi = 0
            for elem in ds.iterall():
                if "CTDIvol" in str(elem.name):
                    try:
                        ctdi = float(elem.value)
                    except:
                        pass
            img = get_image(ds)
            display_image(img, name, date, ctdi)
            save_data(name, date, ctdi)
        except Exception as e:
            print("Failed to load:", e)

def get_image(ds):
    try:
        arr = ds.pixel_array
        image = Image.fromarray(arr)
        return image.resize((256, 256))
    except:
        return Image.new("L", (256, 256), color=128)

def display_image(img, name, date, ctdi):
    frame = tk.Frame(images_frame, bg="#ffffff", bd=2, relief="groove")
    frame.pack(side="left", padx=10, pady=10)
    imgtk = ImageTk.PhotoImage(img.convert("RGB"))
    lbl_img = tk.Label(frame, image=imgtk)
    lbl_img.image = imgtk
    lbl_img.pack()
    tk.Label(frame, text=f"Name: {name}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"Date: {format_date(date)}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 10), bg="#ffffff", fg="green").pack()

def format_date(d):
    try:
        return datetime.strptime(d, "%Y%m%d").strftime("%d-%m-%Y")
    except:
        return "Unknown"

def save_data(name, date, ctdi):
    global data_list
    data_list.append({
        "PatientName": str(name),
        "StudyDate": date,
        "CTDIvol": ctdi
    })

def generate_csv_and_hl7():
    global data_list, hl7_messages
    df = pd.DataFrame(data_list)
    df['StudyDate'] = pd.to_datetime(df['StudyDate'], format="%Y%m%d", errors="coerce")
    df.dropna(inplace=True)

    output = []
    hl7_messages.clear()

    for patient in df['PatientName'].unique():
        patient_df = df[df['PatientName'] == patient].sort_values("StudyDate")
        start_date = patient_df['StudyDate'].min()
        end_date = start_date + pd.DateOffset(years=1)
        yearly_df = patient_df[(patient_df['StudyDate'] >= start_date) & (patient_df['StudyDate'] <= end_date)]
        total = yearly_df['CTDIvol'].sum()

        # Save patient HL7 message
        msg = hl7.parse(f"MSH|^~\\&|CT|HOSPITAL|RAD|HOSPITAL|{datetime.now().strftime('%Y%m%d')}||ORM^O01|MSG00001|P|2.3\rPID|||{patient}||\rOBX|1|NM|CTDIvol||{total}|mGy")
        hl7_messages[str(patient)] = msg

        output.append({
            "PatientName": patient,
            "StartDate": start_date.strftime("%Y-%m-%d"),
            "EndDate": end_date.strftime("%Y-%m-%d"),
            "Total_CTDIvol": total
        })

    result_df = pd.DataFrame(output)
    result_df.to_csv("rad.csv", index=False)
    print("Saved to rad.csv")

def search_hl7():
    name = search_entry.get().strip()
    if name in hl7_messages:
        text.delete(1.0, tk.END)
        text.insert(tk.END, str(hl7_messages[name]))
    else:
        text.delete(1.0, tk.END)
        text.insert(tk.END, "No HL7 message found for this patient.")

# GUI setup
root = tk.Tk()
root.title("Smart DICOM Viewer with CTDIvol")
root.geometry("1200x700")

bg_img = Image.open(r"background-medical.jpg").resize((1200, 700))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.pack(pady=10)

images_frame = tk.Frame(root, bg="white")
images_frame.pack(fill="both", expand=True)

bottom_frame = tk.Frame(root, bg="#f0f0f0")
bottom_frame.pack(fill="x", pady=5)

tk.Button(bottom_frame, text="📄 Save CSV + HL7", command=generate_csv_and_hl7, bg="blue", fg="white").pack(side="left", padx=10)
search_entry = tk.Entry(bottom_frame)
search_entry.pack(side="left")
tk.Button(bottom_frame, text="🔍 Find HL7 by Patient", command=search_hl7).pack(side="left", padx=5)

text = tk.Text(bottom_frame, height=5)
text.pack(side="left", fill="x", expand=True, padx=10)

root.mainloop()


TclError: image "pyimage4" doesn't exist

In [12]:
pip install hl7apy


   ---------------------------------------- 0.0/974.7 kB ? eta -:--:--
   ---------------------------------------- 10.2/974.7 kB ? eta -:--:--
   - ------------------------------------- 30.7/974.7 kB 330.3 kB/s eta 0:00:03
   -- ------------------------------------ 71.7/974.7 kB 491.5 kB/s eta 0:00:02
   ----- -------------------------------- 133.1/974.7 kB 718.6 kB/s eta 0:00:02
   ---------- ----------------------------- 245.8/974.7 kB 1.1 MB/s eta 0:00:01
   --------------- ------------------------ 368.6/974.7 kB 1.4 MB/s eta 0:00:01
   ----------------------- ---------------- 573.4/974.7 kB 1.8 MB/s eta 0:00:01
   ------------------------- -------------- 624.6/974.7 kB 1.9 MB/s eta 0:00:01
   ---------------------------------- ----- 849.9/974.7 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 974.7/974.7 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maryam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import os
import pydicom
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pandas as pd
from datetime import datetime
import hl7

# Globals
data_list = []
hl7_messages = {}

def read_dicom_files():
    file_paths = filedialog.askopenfilenames(title="Select DICOM files", filetypes=[("DICOM files", "*.dcm")])
    for file_path in file_paths:
        try:
            ds = pydicom.dcmread(file_path)
            name = getattr(ds, "PatientName", "Unknown")
            date = getattr(ds, "StudyDate", "00000000")
            ctdi = 0
            for elem in ds.iterall():
                if "CTDIvol" in str(elem.name):
                    try:
                        ctdi = float(elem.value)
                    except:
                        pass
            img = get_image(ds)
            display_image(img, name, date, ctdi)
            save_data(name, date, ctdi)
        except Exception as e:
            print("Failed to load:", e)

def get_image(ds):
    try:
        arr = ds.pixel_array
        image = Image.fromarray(arr)
        return image.resize((256, 256))
    except:
        return Image.new("L", (256, 256), color=128)

def display_image(img, name, date, ctdi):
    frame = tk.Frame(images_frame, bg="#ffffff", bd=2, relief="groove")
    frame.pack(side="left", padx=10, pady=10)
    imgtk = ImageTk.PhotoImage(img.convert("RGB"))
    lbl_img = tk.Label(frame, image=imgtk)
    lbl_img.image = imgtk
    lbl_img.pack()
    tk.Label(frame, text=f"Name: {name}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"Date: {format_date(date)}", font=("Arial", 10), bg="#ffffff").pack()
    tk.Label(frame, text=f"CTDIvol: {ctdi} mGy", font=("Arial", 10), bg="#ffffff", fg="green").pack()

def format_date(d):
    try:
        return datetime.strptime(d, "%Y%m%d").strftime("%d-%m-%Y")
    except:
        return "Unknown"

def save_data(name, date, ctdi):
    global data_list
    data_list.append({
        "PatientName": str(name),
        "StudyDate": date,
        "CTDIvol": ctdi
    })

def generate_csv_and_hl7():
    global data_list, hl7_messages
    df = pd.DataFrame(data_list)
    df['StudyDate'] = pd.to_datetime(df['StudyDate'], format="%Y%m%d", errors="coerce")
    df.dropna(inplace=True)

    output = []
    hl7_messages.clear()

    for patient in df['PatientName'].unique():
        patient_df = df[df['PatientName'] == patient].sort_values("StudyDate")
        start_date = patient_df['StudyDate'].min()
        end_date = start_date + pd.DateOffset(years=1)
        yearly_df = patient_df[(patient_df['StudyDate'] >= start_date) & (patient_df['StudyDate'] <= end_date)]
        total = yearly_df['CTDIvol'].sum()

        # Save patient HL7 message
        msg = hl7.parse(f"MSH|^~\\&|CT|HOSPITAL|RAD|HOSPITAL|{datetime.now().strftime('%Y%m%d')}||ORM^O01|MSG00001|P|2.3\rPID|||{patient}||\rOBX|1|NM|CTDIvol||{total}|mGy")
        hl7_messages[str(patient)] = msg

        output.append({
            "PatientName": patient,
            "StartDate": start_date.strftime("%Y-%m-%d"),
            "EndDate": end_date.strftime("%Y-%m-%d"),
            "Total_CTDIvol": total
        })

    result_df = pd.DataFrame(output)
    result_df.to_csv("rad.csv", index=False)
    print("Saved to rad.csv")

def search_hl7():
    name = search_entry.get().strip()
    if name in hl7_messages:
        text.delete(1.0, tk.END)
        text.insert(tk.END, str(hl7_messages[name]))
    else:
        text.delete(1.0, tk.END)
        text.insert(tk.END, "No HL7 message found for this patient.")

# GUI setup
root = tk.Tk()
root.title("Smart DICOM Viewer with CTDIvol")
root.geometry("1200x700")

bg_img = Image.open(r"background-medical.jpg").resize((1200, 700))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.pack(pady=10)

images_frame = tk.Frame(root, bg="white")
images_frame.pack(fill="both", expand=True)

bottom_frame = tk.Frame(root, bg="#f0f0f0")
bottom_frame.pack(fill="x", pady=5)

tk.Button(bottom_frame, text="📄 Save CSV + HL7", command=generate_csv_and_hl7, bg="blue", fg="white").pack(side="left", padx=10)
search_entry = tk.Entry(bottom_frame)
search_entry.pack(side="left")
tk.Button(bottom_frame, text="🔍 Find HL7 by Patient", command=search_hl7).pack(side="left", padx=5)

text = tk.Text(bottom_frame, height=5)
text.pack(side="left", fill="x", expand=True, padx=10)

root.mainloop()


TclError: image "pyimage5" doesn't exist

In [21]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pydicom
import pandas as pd
from datetime import datetime
import os

# Globals
data_list = []
image_refs = []  # To keep a reference to image objects to prevent garbage collection
hl7_messages = {}  # Store HL7 messages by patient name

# CSV filename
CSV_FILE = "rad.csv"

# Create the main window
root = tk.Tk()
root.title("DICOM CT Dose Viewer")
root.geometry("1200x700")

# Load background image
bg_photo = ImageTk.PhotoImage(Image.open(r"background-medical.jpg").resize((1200, 700)))
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

# Create a canvas and scrollbar
canvas = tk.Canvas(root, width=1180, height=600, bg='white')
scrollbar = tk.Scrollbar(root, orient="vertical", command=canvas.yview)
scrollable_frame = tk.Frame(canvas, bg='white')

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

canvas.place(x=10, y=50)
scrollbar.place(x=1180, y=50, height=600)

# Read DICOM files
def read_dicom_files():
    file_paths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not file_paths:
        return

    global data_list, hl7_messages
    data_list.clear()
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    grouped_data = {}

    for file_path in file_paths:
        try:
            dcm = pydicom.dcmread(file_path)
            img = dcm.pixel_array

            name = dcm.PatientName.family_name if hasattr(dcm.PatientName, 'family_name') else str(dcm.PatientName)
            date = datetime.strptime(dcm.StudyDate, "%Y%m%d") if hasattr(dcm, 'StudyDate') else None
            dose = float(dcm.CTDIvol) if hasattr(dcm, 'CTDIvol') else 0.0

            # Group by patient
            if name not in grouped_data:
                grouped_data[name] = []
            grouped_data[name].append({"date": date, "dose": dose, "file": file_path})

        except Exception as e:
            print(f"Failed to read {file_path}: {e}")

    # Build output
    for name, entries in grouped_data.items():
        total_dose = sum(e['dose'] for e in entries)
        for e in entries:
            frame = tk.Frame(scrollable_frame, bg='white', pady=10)
            frame.pack(side="top", fill="x")

            img_data = pydicom.dcmread(e['file']).pixel_array
            image = ImageTk.PhotoImage(Image.fromarray(img_data).resize((256, 256)))
            image_refs.append(image)  # Keep reference

            panel = tk.Label(frame, image=image, bg='white')
            panel.pack(side="left")

            info = f"Name: {name}\nDate: {e['date'].strftime('%Y-%m-%d')}\nCTDIvol: {e['dose']}"
            label = tk.Label(frame, text=info, font=("Arial", 12), justify="left", bg='white')
            label.pack(side="left", padx=20)

            # Store to CSV data list
            data_list.append([name, e['date'].strftime('%Y-%m-%d'), e['dose']])

        # Save HL7 message
        hl7_messages[name] = f"MSH|^~\\&|CTApp|MedDept|HIS|Hospital|{datetime.now().strftime('%Y%m%d%H%M')}||ORU^R01|{name}|P|2.3.1\nPID|||{name}||{name}||\nOBR|1|||CT\nOBX|1|NM|CTDIvol||{total_dose}|mGy"

    # Save to CSV
    df = pd.DataFrame(data_list, columns=["Name", "Date", "CTDIvol"])
    df.to_csv(CSV_FILE, index=False)
    print("Saved to rad.csv")

# Search and display HL7
def search_hl7():
    search_name = entry_search.get()
    msg = hl7_messages.get(search_name, "No HL7 message found.")
    tk.messagebox.showinfo("HL7 Message", msg)

# UI Elements
btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.place(x=20, y=10)

entry_search = tk.Entry(root, font=("Arial", 12))
entry_search.place(x=300, y=15, width=200)

btn_search = tk.Button(root, text="🔍 Search HL7", command=search_hl7, font=("Arial", 12), bg="blue", fg="white")
btn_search.place(x=510, y=10)

root.mainloop()


Saved to rad.csv


In [18]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import pydicom
import pandas as pd
from datetime import datetime
import os

# Globals
data_list = []
image_refs = []  # To keep a reference to image objects to prevent garbage collection
hl7_messages = {}  # Store HL7 messages by patient name

# CSV filename
CSV_FILE = "rad.csv"

# Create the main window
root = tk.Tk()
root.title("DICOM CT Dose Viewer")
root.geometry("1200x700")

# Load background image
bg_photo = ImageTk.PhotoImage(Image.open("background-medical.jpg").resize((1200, 700)))
# bg_label = tk.Label(root, image=bg_photo)
# bg_label.place(x=0, y=0, relwidth=1, relheight=1)

# Create a canvas and scrollbar
canvas = tk.Canvas(root, width=1180, height=600, bg='white')
scrollbar = tk.Scrollbar(root, orient="vertical", command=canvas.yview)
scrollable_frame = tk.Frame(canvas, bg='white')

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

canvas.place(x=10, y=50)
scrollbar.place(x=1180, y=50, height=600)

# Read DICOM files
def read_dicom_files():
    file_paths = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not file_paths:
        return

    global data_list, hl7_messages
    data_list.clear()
    for widget in scrollable_frame.winfo_children():
        widget.destroy()

    grouped_data = {}

    for file_path in file_paths:
        try:
            dcm = pydicom.dcmread(file_path)
            img = dcm.pixel_array

            name = dcm.PatientName.family_name if hasattr(dcm.PatientName, 'family_name') else str(dcm.PatientName)
            date = datetime.strptime(dcm.StudyDate, "%Y%m%d") if hasattr(dcm, 'StudyDate') else None
            dose = float(dcm.CTDIvol) if hasattr(dcm, 'CTDIvol') else 0.0

            # Group by patient
            if name not in grouped_data:
                grouped_data[name] = []
            grouped_data[name].append({"date": date, "dose": dose, "file": file_path})

        except Exception as e:
            print(f"Failed to read {file_path}: {e}")

    # Build output
    for name, entries in grouped_data.items():
        total_dose = sum(e['dose'] for e in entries)
        for e in entries:
            frame = tk.Frame(scrollable_frame, bg='white', pady=10)
            frame.pack(side="top", fill="x")

            img_data = pydicom.dcmread(e['file']).pixel_array
            image = ImageTk.PhotoImage(Image.fromarray(img_data).resize((256, 256)))
            image_refs.append(image)  # Keep reference

            panel = tk.Label(frame, image=image, bg='white')
            panel.pack(side="left")

            info = f"Name: {name}\nDate: {e['date'].strftime('%Y-%m-%d')}\nCTDIvol: {e['dose']}"
            label = tk.Label(frame, text=info, font=("Arial", 12), justify="left", bg='white')
            label.pack(side="left", padx=20)

            # Store to CSV data list
            data_list.append([name, e['date'].strftime('%Y-%m-%d'), e['dose']])

        # Save HL7 message
        hl7_messages[name] = f"MSH|^~\\&|CTApp|MedDept|HIS|Hospital|{datetime.now().strftime('%Y%m%d%H%M')}||ORU^R01|{name}|P|2.3.1\nPID|||{name}||{name}||\nOBR|1|||CT\nOBX|1|NM|CTDIvol||{total_dose}|mGy"

    # Save to CSV
    df = pd.DataFrame(data_list, columns=["Name", "Date", "CTDIvol"])
    df.to_csv(CSV_FILE, index=False)
    print("Saved to rad.csv")

# Search and display HL7
def search_hl7():
    search_name = entry_search.get()
    msg = hl7_messages.get(search_name, "No HL7 message found.")
    tk.messagebox.showinfo("HL7 Message", msg)

# UI Elements
btn = tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files, font=("Arial", 14), bg="green", fg="white")
btn.place(x=20, y=10)

entry_search = tk.Entry(root, font=("Arial", 12))
entry_search.place(x=300, y=15, width=200)

btn_search = tk.Button(root, text="🔍 Search HL7", command=search_hl7, font=("Arial", 12), bg="blue", fg="white")
btn_search.place(x=510, y=10)

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\maryam\AppData\Local\Temp\ipykernel_13180\2279200120.py", line 80, in read_dicom_files
    image = ImageTk.PhotoImage(Image.fromarray(img_data).resize((256, 256)))
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\maryam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\PIL\ImageTk.py", line 129, in __init__
    self.__photo = tkinter.PhotoImage(**kw)
                   ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\tkinter\__init__.py", line 4150, in __init__
    Image.__init__(self, 'photo', name

In [20]:
import tkinter as tk
from tkinter import filedialog, Scrollbar, Canvas
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

def convert_to_hl7(patient_name, date, ctdi):
    return f"MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3\nPID|||{patient_name}|||\nOBR|||{date}|||CT\nOBX|||CTDIvol||{ctdi}|mGy"

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files: return

    all_data = []
    dose_summary = {}

    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        ds = pydicom.dcmread(path)
        name = str(getattr(ds, "PatientName", "Unknown"))
        ctdi = float(getattr(ds, "CTDIvol", 0))
        date_str = getattr(ds, "StudyDate", "00000000")
        try:
            date_obj = datetime.strptime(date_str, "%Y%m%d")
        except:
            date_obj = datetime.now()

        # عرض الصورة
        img = Image.fromarray(ds.pixel_array)
        img.thumbnail((300, 300))
        photo = ImageTk.PhotoImage(img)

        frame = tk.Frame(content_frame, bg="white", bd=2, relief="groove")
        frame.pack(side="left", padx=10, pady=10)

        img_label = tk.Label(frame, image=photo)
        img_label.image = photo
        img_label.pack()

        info = f"Name: {name}\nDate: {date_obj.date()}\nCTDIvol: {ctdi} mGy"
        tk.Label(frame, text=info, bg="white", font=("Arial", 10)).pack()

        all_data.append({"Name": name, "Date": date_obj.date(), "CTDIvol": ctdi})

        hl7_msg = convert_to_hl7(name, date_obj.strftime("%Y%m%d"), ctdi)
        with open(f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7", "w") as f:
            f.write(hl7_msg)

        # إجمالي الجرعة لكل مريض
        dose_summary[name] = dose_summary.get(name, 0) + ctdi

    # حفظ CSV
    df = pd.DataFrame(all_data)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    for name, total in dose_summary.items():
        tk.Label(content_frame, text=f"Total dose for {name}: {total:.2f} mGy",
                 font=("Arial", 12, "bold"), bg="#e0f7fa").pack(side="left", padx=10)

# نافذة البرنامج
root = tk.Tk()
root.title("DICOM Viewer + HL7 + CSV")
root.geometry("1200x700")

# خلفية
bg_image = Image.open("background-medical.jpg").resize((1200, 700))
bg_photo = ImageTk.PhotoImage(bg_image)
bg_label = tk.Label(root, image=bg_photo)
bg_label.image = bg_photo
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

# Canvas مع شريط تمرير
canvas = Canvas(root, bg="white", width=1150, height=600)
canvas.place(x=25, y=80)
scrollbar = Scrollbar(root, orient="horizontal", command=canvas.xview)
scrollbar.place(x=25, y=680, width=1150)
canvas.configure(xscrollcommand=scrollbar.set)

content_frame = tk.Frame(canvas, bg="white")
canvas.create_window((0, 0), window=content_frame, anchor="nw")
content_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

# زر اختيار الصور
tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files,
          font=("Arial", 14), bg="green", fg="white").place(x=500, y=20)

root.mainloop()


In [22]:
import tkinter as tk
from tkinter import filedialog, Scrollbar, Canvas, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

def convert_to_hl7(patient_name, date, ctdi):
    return f"MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3\nPID|||{patient_name}|||\nOBR|||{date}|||CT\nOBX|||CTDIvol||{ctdi}|mGy"

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data = []
    dose_summary = {}

    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            # عرض الصورة
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((300, 300))
                photo = ImageTk.PhotoImage(img)

                frame = tk.Frame(content_frame, bg="white", bd=2, relief="groove")
                frame.pack(side="left", padx=10, pady=10)

                img_label = tk.Label(frame, image=photo)
                img_label.image = photo
                img_label.pack()
            else:
                frame = tk.Frame(content_frame, bg="white", bd=2, relief="groove")
                frame.pack(side="left", padx=10, pady=10)
                tk.Label(frame, text="No Image Data", bg="white", font=("Arial", 10)).pack()

            info = f"Name: {name}\nDate: {date_obj.date()}\nCTDIvol: {ctdi} mGy"
            tk.Label(frame, text=info, bg="white", font=("Arial", 10)).pack()

            all_data.append({"Name": name, "Date": date_obj.date(), "CTDIvol": ctdi})

            hl7_msg = convert_to_hl7(name, date_obj.strftime("%Y%m%d"), ctdi)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

            # إجمالي الجرعة لكل مريض
            dose_summary[name] = dose_summary.get(name, 0) + ctdi

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    # حفظ CSV
    df = pd.DataFrame(all_data)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    for name, total in dose_summary.items():
        tk.Label(content_frame, text=f"Total dose for {name}: {total:.2f} mGy",
                 font=("Arial", 12, "bold"), bg="#e0f7fa").pack(side="left", padx=10)

def search_hl7_message():
    name = search_entry.get().strip()
    if not name:
        messagebox.showwarning("Input Error", "Please enter a patient name.")
        return

    # البحث عن ملفات HL7 التي تحتوي على اسم المريض
    found = False
    for filename in os.listdir(HL7_DIR):
        if name.lower() in filename.lower():
            filepath = os.path.join(HL7_DIR, filename)
            with open(filepath, "r") as f:
                hl7_content = f.read()
            hl7_text.delete("1.0", tk.END)
            hl7_text.insert(tk.END, hl7_content)
            found = True
            break

    if not found:
        messagebox.showinfo("Not Found", f"No HL7 message found for patient: {name}")
        hl7_text.delete("1.0", tk.END)

# نافذة البرنامج
root = tk.Tk()
root.title("DICOM Viewer + HL7 + CSV")
root.geometry("1200x800")

# خلفية
try:
    bg_image = Image.open("background.jpg").resize((1200, 800))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = tk.Label(root, image=bg_photo)
    bg_label.image = bg_photo
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Background image not found or failed to load: {e}")

# Canvas مع شريط تمرير
canvas = Canvas(root, bg="white", width=1150, height=500)
canvas.place(x=25, y=150)
scrollbar = Scrollbar(root, orient="horizontal", command=canvas.xview)
scrollbar.place(x=25, y=650, width=1150)
canvas.configure(xscrollcommand=scrollbar.set)

content_frame = tk.Frame(canvas, bg="white")
canvas.create_window((0, 0), window=content_frame, anchor="nw")
content_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

# زر اختيار الصور
tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files,
          font=("Arial", 14), bg="green", fg="white").place(x=500, y=20)

# حقل البحث
tk.Label(root, text="🔍 Search HL7 by Patient Name:", font=("Arial", 12), bg="lightblue").place(x=50, y=80)
search_entry = tk.Entry(root, font=("Arial", 12), width=30)
search_entry.place(x=300, y=80)
tk.Button(root, text="Search", command=search_hl7_message,
          font=("Arial", 12), bg="blue", fg="white").place(x=550, y=75)

# مربع نص لعرض رسالة HL7
hl7_text = tk.Text(root, height=10, width=100, font=("Courier", 10))
hl7_text.place(x=50, y=700)

root.mainloop()


Background image not found or failed to load: [Errno 2] No such file or directory: 'background.jpg'


In [23]:
pip install pydicom pandas pillow customtkinter


   ---------------------------------------- 0.0/296.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/296.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/296.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/296.1 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/296.1 kB 330.3 kB/s eta 0:00:01
   ----- --------------------------------- 41.0/296.1 kB 245.8 kB/s eta 0:00:02
   ------------ -------------------------- 92.2/296.1 kB 438.1 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/296.1 kB 554.9 kB/s eta 0:00:01
   -------------------------------- ----- 256.0/296.1 kB 927.4 kB/s eta 0:00:01
   -------------------------------------- 296.1/296.1 kB 916.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\maryam\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [27]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime

# إعدادات الواجهة
ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

# المسارات
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

# الوظائف
def convert_to_hl7(patient_name, date, ctdi):
    return f"MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3\nPID|||{patient_name}|||\nOBR|||{date}|||CT\nOBX|||CTDIvol||{ctdi}|mGy"

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014  # تحويل mGy إلى mSv

            # إنشاء الصورة المصغّرة
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                photo = ImageTk.PhotoImage(img)
            else:
                photo = None

            # حفظ البيانات
            all_data.append({
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Image": photo
            })

            # حفظ رسالة HL7
            hl7_msg = convert_to_hl7(name, date_obj.strftime("%Y%m%d"), ctdi)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    # حفظ CSV
    df = pd.DataFrame(all_data)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    display_images()

def display_images():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # فرز البيانات حسب الخيار المحدد
    sort_option = sort_var.get()
    if sort_option == "Name":
        sorted_data = sorted(all_data, key=lambda x: x["Name"])
    else:
        sorted_data = sorted(all_data, key=lambda x: x["Date"])

    # عرض الصور والمعلومات
    columns = 3
    for index, data in enumerate(sorted_data):
        row = index // columns
        col = index % columns

        frame = ctk.CTkFrame(content_frame, width=220, height=300)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)
        else:
            img_label = ctk.CTkLabel(frame, text="No Image Data")
            img_label.pack(pady=5)

        info = (
            f"Name: {data['Name']}\n"
            f"Date: {data['Date'].date()}\n"
            f"CTDIvol: {data['CTDIvol']} mGy\n"
            f"DLP: {data['DLP']} mGy·cm\n"
            f"Dose: {data['mSv']:.2f} mSv"
        )
        info_label = ctk.CTkLabel(frame, text=info, justify="left")
        info_label.pack(pady=5)

def search_hl7_message():
    name = search_entry.get().strip()
    if not name:
        messagebox.showwarning("Input Error", "Please enter a patient name.")
        return

    # البحث عن ملفات HL7 التي تحتوي على اسم المريض
    found = False
    for filename in os.listdir(HL7_DIR):
        if name.lower() in filename.lower():
            filepath = os.path.join(HL7_DIR, filename)
            with open(filepath, "r") as f:
                hl7_content = f.read()
            hl7_textbox.delete("1.0", "end")
            hl7_textbox.insert("end", hl7_content)
            found = True
            break

    if not found:
        messagebox.showinfo("Not Found", f"No HL7 message found for patient: {name}")
        hl7_textbox.delete("1.0", "end")

# إنشاء الواجهة
root = ctk.CTk()
root.title("DICOM Viewer + HL7 + CSV")
root.geometry("1200x800")

# خلفية
try:
    bg_image = Image.open("1020.jpg").resize((1200, 800))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = ctk.CTkLabel(root, image=bg_photo, text="")
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Background image not found or failed to load: {e}")

# إطار المحتوى
content_frame = ctk.CTkScrollableFrame(root, width=1150, height=500)
content_frame.place(x=25, y=150)

# زر اختيار الملفات
select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(x=50, y=20)

# حقل البحث
search_label = ctk.CTkLabel(root, text="🔍 Search HL7 by Patient Name:")
search_label.place(x=300, y=20)
search_entry = ctk.CTkEntry(root, width=200)
search_entry.place(x=500, y=20)
search_button = ctk.CTkButton(root, text="Search", command=search_hl7_message)
search_button.place(x=710, y=20)

# خيار الفرز
sort_var = ctk.StringVar(value="Date")
sort_label = ctk.CTkLabel(root, text="Sort by:")
sort_label.place(x=900, y=20)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_images())
sort_optionmenu.place(x=970, y=20)

# مربع نص لعرض رسالة HL7
hl7_textbox = ctk.CTkTextbox(root, width=1100, height=100)
hl7_textbox.place(x=50, y=670)

# قائمة البيانات
all_data = []

root.mainloop()


In [28]:
import tkinter as tk
from tkinter import filedialog, Scrollbar, Canvas, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta

CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

# دالة لتحويل CTDIvol إلى HL7
def convert_to_hl7(patient_name, date, ctdi):
    return f"MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3\nPID|||{patient_name}|||\nOBR|||{date}|||CT\nOBX|||CTDIvol||{ctdi}|mGy"

# قراءة ملفات DICOM
def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data = []
    dose_summary = {}

    # مسح الصور الحالية من الواجهة
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            # عرض الصورة
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))  # تصغير الصور
                photo = ImageTk.PhotoImage(img)

                frame = tk.Frame(content_frame, bg="white", bd=2, relief="groove")
                frame.pack(side="left", padx=10, pady=10)

                img_label = tk.Label(frame, image=photo)
                img_label.image = photo
                img_label.pack()
            else:
                frame = tk.Frame(content_frame, bg="white", bd=2, relief="groove")
                frame.pack(side="left", padx=10, pady=10)
                tk.Label(frame, text="No Image Data", bg="white", font=("Arial", 10)).pack()

            info = f"Name: {name}\nDate: {date_obj.date()}\nCTDIvol: {ctdi} mGy"
            tk.Label(frame, text=info, bg="white", font=("Arial", 10)).pack()

            all_data.append({"Name": name, "Date": date_obj.date(), "CTDIvol": ctdi})

            hl7_msg = convert_to_hl7(name, date_obj.strftime("%Y%m%d"), ctdi)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

            # إجمالي الجرعة لكل مريض
            dose_summary[name] = dose_summary.get(name, 0) + ctdi

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    # حفظ CSV
    df = pd.DataFrame(all_data)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    for name, total in dose_summary.items():
        tk.Label(content_frame, text=f"Total dose for {name}: {total:.2f} mGy",
                 font=("Arial", 12, "bold"), bg="#e0f7fa").pack(side="left", padx=10)

# دالة لحساب مجموع الجرعة لكل اسم خلال سنة واحدة
def calculate_total_dose():
    df = pd.read_csv(CSV_FILE, parse_dates=["Date"])
    df["Date"] = pd.to_datetime(df["Date"])

    results = []
    for name, group in df.groupby("Name"):
        group = group.sort_values("Date").reset_index(drop=True)
        i = 0
        while i < len(group):
            start = group.loc[i, "Date"]
            end = start + timedelta(days=365)
            in_window = group[(group["Date"] >= start) & (group["Date"] <= end)]
            total = in_window["CTDIvol"].sum()
            results.append({
                "Name": name,
                "From": start.date(),
                "To": in_window["Date"].max().date(),
                "Total_CTDIvol (mGy)": total,
                "Dose_in_mSv": round(total * 0.017, 2)  # تحويل من mGy إلى mSv
            })
            i = in_window.index[-1] + 1

    summary_df = pd.DataFrame(results)
    for row in summary_df.itertuples():
        info = f"Name: {row.Name}, From: {row.From}, To: {row.To}, Total Dose: {row.Total_CTDIvol} mGy, Dose in mSv: {row.Dose_in_mSv}"
        tk.Label(content_frame, text=info, font=("Arial", 10), bg="lightyellow").pack(pady=5)

# البحث عن رسائل HL7
def search_hl7_message():
    name = search_entry.get().strip()
    if not name:
        messagebox.showwarning("Input Error", "Please enter a patient name.")
        return

    found = False
    for filename in os.listdir(HL7_DIR):
        if name.lower() in filename.lower():
            filepath = os.path.join(HL7_DIR, filename)
            with open(filepath, "r") as f:
                hl7_content = f.read()
            hl7_text.delete("1.0", tk.END)
            hl7_text.insert(tk.END, hl7_content)
            found = True
            break

    if not found:
        messagebox.showinfo("Not Found", f"No HL7 message found for patient: {name}")
        hl7_text.delete("1.0", tk.END)

# نافذة البرنامج
root = tk.Tk()
root.title("DICOM Viewer + HL7 + CSV")
root.geometry("1200x800")

# خلفية
try:
    bg_image = Image.open("background.jpg").resize((1200, 800))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = tk.Label(root, image=bg_photo)
    bg_label.image = bg_photo
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Background image not found or failed to load: {e}")

# Canvas مع شريط تمرير
canvas = Canvas(root, bg="white", width=1150, height=500)
canvas.place(x=25, y=150)
scrollbar = Scrollbar(root, orient="horizontal", command=canvas.xview)
scrollbar.place(x=25, y=650, width=1150)
canvas.configure(xscrollcommand=scrollbar.set)

content_frame = tk.Frame(canvas, bg="white")
canvas.create_window((0, 0), window=content_frame, anchor="nw")
content_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

# زر اختيار الصور
tk.Button(root, text="📂 Select DICOM Files", command=read_dicom_files,
          font=("Arial", 14), bg="green", fg="white").place(x=500, y=20)

# زر حساب الجرعة الكلية
tk.Button(root, text="📊 Calculate Total Dose", command=calculate_total_dose,
          font=("Arial", 14), bg="blue", fg="white").place(x=350, y=20)

# حقل البحث
tk.Label(root, text="🔍 Search HL7 by Patient Name:", font=("Arial", 12), bg="lightblue").place(x=50, y=80)
search_entry = tk.Entry(root, font=("Arial", 12), width=30)
search_entry.place(x=300, y=80)
tk.Button(root, text="Search", command=search_hl7_message,
          font=("Arial", 12), bg="blue", fg="white").place(x=550, y=75)

# مربع نص لعرض رسالة HL7
hl7_text = tk.Text(root, height=10, width=100, font=("Courier", 10))
hl7_text.place(x=50, y=700)

root.mainloop()


Background image not found or failed to load: [Errno 2] No such file or directory: 'background.jpg'


In [29]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta

# إعدادات الواجهة
ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

# المسارات
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

# الوظائف
def convert_to_hl7(patient_name, date, ctdi):
    return f"MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3\nPID|||{patient_name}|||\nOBR|||{date}|||CT\nOBX|||CTDIvol||{ctdi}|mGy"

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014  # تحويل mGy إلى mSv

            # إنشاء الصورة المصغّرة
            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                photo = ImageTk.PhotoImage(img)
            else:
                photo = None

            # حفظ البيانات
            all_data.append({
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Image": photo
            })

            # حفظ رسالة HL7
            hl7_msg = convert_to_hl7(name, date_obj.strftime("%Y%m%d"), ctdi)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    # حفظ CSV
    df = pd.DataFrame(all_data)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    display_images()

def display_images():
    for widget in content_frame.winfo_children():
        widget.destroy()

    # فرز البيانات حسب الخيار المحدد
    sort_option = sort_var.get()
    if sort_option == "Name":
        sorted_data = sorted(all_data, key=lambda x: x["Name"])
    else:
        sorted_data = sorted(all_data, key=lambda x: x["Date"])

    # عرض الصور والمعلومات
    columns = 3
    for index, data in enumerate(sorted_data):
        row = index // columns
        col = index % columns

        frame = ctk.CTkFrame(content_frame, width=220, height=300)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)
        else:
            img_label = ctk.CTkLabel(frame, text="No Image Data")
            img_label.pack(pady=5)

        info = (
            f"Name: {data['Name']}\n"
            f"Date: {data['Date'].date()}\n"
            f"CTDIvol: {data['CTDIvol']} mGy\n"
            f"DLP: {data['DLP']} mGy·cm\n"
            f"Dose: {data['mSv']:.2f} mSv"
        )
        info_label = ctk.CTkLabel(frame, text=info, justify="left")
        info_label.pack(pady=5)

def search_hl7_message():
    name = search_entry.get().strip()
    if not name:
        messagebox.showwarning("Input Error", "Please enter a patient name.")
        return

    # البحث عن ملفات HL7 التي تحتوي على اسم المريض
    found = False
    for filename in os.listdir(HL7_DIR):
        if name.lower() in filename.lower():
            filepath = os.path.join(HL7_DIR, filename)
            with open(filepath, "r") as f:
                hl7_content = f.read()
            hl7_textbox.delete("1.0", "end")
            hl7_textbox.insert("end", hl7_content)
            found = True
            break

    if not found:
        messagebox.showinfo("Not Found", f"No HL7 message found for patient: {name}")
        hl7_textbox.delete("1.0", "end")

def clear_images():
    # مسح الصور المعروضة في الواجهة
    for widget in content_frame.winfo_children():
        widget.destroy()

def calculate_total_dose():
    df = pd.read_csv(CSV_FILE, parse_dates=["Date"])
    df["Date"] = pd.to_datetime(df["Date"])

    results = []
    for name, group in df.groupby("Name"):
        group = group.sort_values("Date").reset_index(drop=True)
        i = 0
        while i < len(group):
            start = group.loc[i, "Date"]
            end = start + timedelta(days=365)
            in_window = group[(group["Date"] >= start) & (group["Date"] <= end)]
            total = in_window["CTDIvol"].sum()
            results.append({
                "Name": name,
                "From": start.date(),
                "To": in_window["Date"].max().date(),
                "Total_CTDIvol (mGy)": total,
                "Dose_in_mSv": round(total * 0.017, 2)  # تحويل من mGy إلى mSv
            })
            i = in_window.index[-1] + 1

    summary_df = pd.DataFrame(results)
    for row in summary_df.itertuples():
        info = f"Name: {row.Name}, From: {row.From}, To: {row.To}, Total Dose: {row.Total_CTDIvol} mGy, Dose in mSv: {row.Dose_in_mSv}"
        tk.Label(content_frame, text=info, font=("Arial", 10), bg="lightyellow").pack(pady=5)

# إنشاء الواجهة
root = ctk.CTk()
root.title("DICOM Viewer + HL7 + CSV")
root.geometry("1200x800")

# خلفية
try:
    bg_image = Image.open("1020.jpg").resize((1200, 800))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = ctk.CTkLabel(root, image=bg_photo, text="")  # استخدام خلفية
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Background image not found or failed to load: {e}")

# إطار المحتوى
content_frame = ctk.CTkScrollableFrame(root, width=1150, height=500)
content_frame.place(x=25, y=150)

# زر اختيار الملفات
select_button = ctk.CTkButton(root, text="📂 Select DICOM Files", command=read_dicom_files)
select_button.place(x=50, y=20)

# زر مسح الصور
clear_button = ctk.CTkButton(root, text="🗑️ Clear Images", command=clear_images)
clear_button.place(x=250, y=20)

# حقل البحث
search_label = ctk.CTkLabel(root, text="🔍 Search HL7 by Patient Name:")
search_label.place(x=300, y=20)
search_entry = ctk.CTkEntry(root, width=200)
search_entry.place(x=500, y=20)
search_button = ctk.CTkButton(root, text="Search", command=search_hl7_message)
search_button.place(x=710, y=20)

# خيار الفرز
sort_var = ctk.StringVar(value="Date")
sort_label = ctk.CTkLabel(root, text="Sort by:")
sort_label.place(x=900, y=20)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_images())
sort_optionmenu.place(x=970, y=20)

# مربع نص لعرض رسالة HL7
hl7_textbox = ctk.CTkTextbox(root, width=1100, height=100)
hl7_textbox.place(x=50, y=670)

# زر حساب المجموع الكلي
calculate_button = ctk.CTkButton(root, text="📊 Calculate Total Dose", command=calculate_total_dose)
calculate_button.place(x=850, y=20)

# قائمة البيانات
all_data = []

root.mainloop()


In [1]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

# إعدادات الواجهة
ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []

# تحويل HL7
def convert_to_hl7(patient_name, date, ctdi):
    return f"MSH|^~\\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3\nPID|||{patient_name}|||\nOBR|||{date}|||CT\nOBX|||CTDIvol||{ctdi}|mGy"

# اختيار الملفات
def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                photo = ImageTk.PhotoImage(img)
            else:
                photo = None

            all_data.append({
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Image": photo
            })

            hl7_msg = convert_to_hl7(name, date_obj.strftime("%Y%m%d"), ctdi)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    df = pd.DataFrame(all_data)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    display_images()

# عرض الصور والبيانات
def display_images():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])

    dose_summary = defaultdict(lambda: defaultdict(float))

    columns = 3
    for index, data in enumerate(sorted_data):
        row = index // columns
        col = index % columns

        frame = ctk.CTkFrame(content_frame, width=250, height=350)
        frame.grid(row=row, column=col, padx=10, pady=10)

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)
        else:
            ctk.CTkLabel(frame, text="No Image").pack()

        info = (
            f"Name: {data['Name']}\n"
            f"Date: {data['Date'].date()}\n"
            f"CTDIvol: {data['CTDIvol']} mGy\n"
            f"DLP: {data['DLP']} mGy·cm\n"
            f"Dose: {data['mSv']:.2f} mSv"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)

        # حساب مجموع السنة
        start_date = data["Date"]
        end_date = start_date + timedelta(days=365)
        patient_name = data["Name"]
        for d in all_data:
            if d["Name"] == patient_name and start_date <= d["Date"] <= end_date:
                dose_summary[patient_name][start_date.year] += d["mSv"]

    # عرض الملخص
    summary_box.delete("1.0", "end")
    for name in dose_summary:
        for year, total_dose in dose_summary[name].items():
            summary_box.insert("end", f"{name} - Year starting {year}: Total mSv = {total_dose:.2f}\n")

# بحث HL7
def search_hl7_message():
    name = search_entry.get().strip()
    if not name:
        messagebox.showwarning("Input Error", "Please enter a patient name.")
        return

    for filename in os.listdir(HL7_DIR):
        if name.lower() in filename.lower():
            filepath = os.path.join(HL7_DIR, filename)
            with open(filepath, "r") as f:
                hl7_content = f.read()
            hl7_textbox.delete("1.0", "end")
            hl7_textbox.insert("end", hl7_content)
            return

    messagebox.showinfo("Not Found", f"No HL7 message found for patient: {name}")
    hl7_textbox.delete("1.0", "end")

# مسح الصور
def clear_images():
    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()
    summary_box.delete("1.0", "end")
    hl7_textbox.delete("1.0", "end")

# إنشاء الواجهة
root = ctk.CTk()
root.title("DICOM Viewer + HL7 + Summary")
root.geometry("1250x850")

try:
    bg_image = Image.open("g.jpg").resize((1250, 850))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = ctk.CTkLabel(root, image=bg_photo, text="")
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Background image not loaded: {e}")

# رأس الصفحة
# ctk.CTkLabel(root, text="📁 DICOM & HL7 Viewer", font=("Arial", 24)).place(x=30, y=10)

select_button = ctk.CTkButton(root, text="Select DICOM Files", command=read_dicom_files)
select_button.place(x=30, y=60)

clear_button = ctk.CTkButton(root, text="🗑️ Clear All", command=clear_images)
clear_button.place(x=180, y=60)

ctk.CTkLabel(root, text="🔍 Search HL7 by Name:").place(x=400, y=60)
search_entry = ctk.CTkEntry(root, width=180)
search_entry.place(x=580, y=60)
search_button = ctk.CTkButton(root, text="Search", command=search_hl7_message)
search_button.place(x=770, y=60)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(x=900, y=60)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_images())
sort_optionmenu.place(x=970, y=60)

# إطار الصور
content_frame = ctk.CTkScrollableFrame(root, width=1180, height=500)
content_frame.place(x=30, y=110)

# عرض HL7
hl7_textbox = ctk.CTkTextbox(root, width=1180, height=100)
hl7_textbox.place(x=30, y=630)

# ملخص الجرعة
summary_box = ctk.CTkTextbox(root, width=1180, height=60)
summary_box.place(x=30, y=740)

root.mainloop()


C:\Users\maryam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


In [2]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import pydicom
import os
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")
CSV_FILE = "rad.csv"
HL7_DIR = "hl7_messages"
os.makedirs(HL7_DIR, exist_ok=True)

all_data = []

# تحويل HL7

def convert_to_hl7(ds, msv):
    name = str(getattr(ds, "PatientName", "Unknown"))
    date = getattr(ds, "StudyDate", "00000000")
    ctdi = float(getattr(ds, "CTDIvol", 0))
    dlp = float(getattr(ds, "DLP", 0))
    gender = getattr(ds, "PatientSex", "")
    dob = getattr(ds, "PatientBirthDate", "")
    study_id = getattr(ds, "StudyID", "")
    accession = getattr(ds, "AccessionNumber", "")

    return f"""MSH|^~\&|CTApp|Hospital|PACS|Hospital|{date}||ORU^R01|MSG00001|P|2.3
PID|||{name}||{dob}|{gender}||
OBR|||{study_id}^{accession}|||CT
OBX|1|NM|CTDIvol||{ctdi}|mGy|||
OBX|2|NM|DLP||{dlp}|mGy*cm|||
OBX|3|NM|EffectiveDose||{msv:.2f}|mSv|||"""

# اختيار الملفات

def read_dicom_files():
    files = filedialog.askopenfilenames(filetypes=[("DICOM files", "*.dcm")])
    if not files:
        return

    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()

    for path in files:
        try:
            ds = pydicom.dcmread(path)
            name = str(getattr(ds, "PatientName", "Unknown"))
            ctdi = float(getattr(ds, "CTDIvol", 0))
            dlp = float(getattr(ds, "DLP", 0))
            date_str = getattr(ds, "StudyDate", "00000000")
            try:
                date_obj = datetime.strptime(date_str, "%Y%m%d")
            except:
                date_obj = datetime.now()

            msv = ctdi * 0.014

            if 'PixelData' in ds:
                img = Image.fromarray(ds.pixel_array)
                img.thumbnail((200, 200))
                photo = ImageTk.PhotoImage(img)
            else:
                photo = None

            data_dict = {
                "Name": name,
                "Date": date_obj,
                "CTDIvol": ctdi,
                "DLP": dlp,
                "mSv": msv,
                "Image": photo,
                "Sex": getattr(ds, "PatientSex", ""),
                "DOB": getattr(ds, "PatientBirthDate", ""),
                "StudyID": getattr(ds, "StudyID", ""),
                "Accession": getattr(ds, "AccessionNumber", "")
            }

            all_data.append(data_dict)

            hl7_msg = convert_to_hl7(ds, msv)
            hl7_filename = f"{HL7_DIR}/{name}_{date_obj.strftime('%Y%m%d')}.hl7"
            with open(hl7_filename, "w") as f:
                f.write(hl7_msg)

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process file {path}.\nError: {e}")

    df = pd.DataFrame(all_data)
    df.drop(columns=["Image"], inplace=True)
    if os.path.exists(CSV_FILE):
        df.to_csv(CSV_FILE, mode='a', index=False, header=False)
    else:
        df.to_csv(CSV_FILE, index=False)

    display_images()

# عرض الصور والبيانات

def display_images():
    for widget in content_frame.winfo_children():
        widget.destroy()

    sort_option = sort_var.get()
    sorted_data = sorted(all_data, key=lambda x: x[sort_option])
    dose_summary = defaultdict(lambda: defaultdict(float))

    columns = 3
    for index, data in enumerate(sorted_data):
        row = index // columns
        col = index % columns

        frame = ctk.CTkFrame(content_frame)
        frame.grid(row=row, column=col, padx=10, pady=10, sticky="nsew")

        if data["Image"]:
            img_label = ctk.CTkLabel(frame, image=data["Image"], text="")
            img_label.image = data["Image"]
            img_label.pack(pady=5)
        else:
            ctk.CTkLabel(frame, text="No Image").pack()

        info = (
            f"Name: {data['Name']}\n"
            f"Date: {data['Date'].date()}\n"
            f"CTDIvol: {data['CTDIvol']} mGy\n"
            f"DLP: {data['DLP']} mGy·cm\n"
            f"Dose: {data['mSv']:.2f} mSv"
        )
        ctk.CTkLabel(frame, text=info, justify="left").pack(pady=5)

        start_date = data["Date"]
        end_date = start_date + timedelta(days=365)
        patient_name = data["Name"]
        for d in all_data:
            if d["Name"] == patient_name and start_date <= d["Date"] <= end_date:
                dose_summary[patient_name][start_date.year] += d["mSv"]

    summary_box.delete("1.0", "end")
    for name in dose_summary:
        for year, total_dose in dose_summary[name].items():
            summary_box.insert("end", f"{name} - Year starting {year}: Total mSv = {total_dose:.2f}\n")

# بحث HL7

def search_hl7_message():
    name = search_entry.get().strip()
    if not name:
        messagebox.showwarning("Input Error", "Please enter a patient name.")
        return

    for filename in os.listdir(HL7_DIR):
        if name.lower() in filename.lower():
            filepath = os.path.join(HL7_DIR, filename)
            with open(filepath, "r") as f:
                hl7_content = f.read()
            hl7_textbox.delete("1.0", "end")
            hl7_textbox.insert("end", hl7_content)
            return

    messagebox.showinfo("Not Found", f"No HL7 message found for patient: {name}")
    hl7_textbox.delete("1.0", "end")

# مسح الصور

def clear_images():
    all_data.clear()
    for widget in content_frame.winfo_children():
        widget.destroy()
    summary_box.delete("1.0", "end")
    hl7_textbox.delete("1.0", "end")

# إنشاء الواجهة
root = ctk.CTk()
root.title("DICOM Viewer + HL7 + Summary")
root.geometry("1250x850")
root.resizable(True, True)

try:
    bg_image = Image.open("g.jpg").resize((1250, 850))
    bg_photo = ImageTk.PhotoImage(bg_image)
    bg_label = ctk.CTkLabel(root, image=bg_photo, text="")
    bg_label.place(relx=0, rely=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Background image not loaded: {e}")

select_button = ctk.CTkButton(root, text="Select DICOM Files", command=read_dicom_files)
select_button.place(relx=0.02, rely=0.05)

clear_button = ctk.CTkButton(root, text="\U0001F5D1 Clear All", command=clear_images)
clear_button.place(relx=0.15, rely=0.05)

ctk.CTkLabel(root, text="\U0001F50D Search HL7 by Name:").place(relx=0.35, rely=0.05)
search_entry = ctk.CTkEntry(root, width=180)
search_entry.place(relx=0.52, rely=0.05)
search_button = ctk.CTkButton(root, text="Search", command=search_hl7_message)
search_button.place(relx=0.68, rely=0.05)

sort_var = ctk.StringVar(value="Date")
ctk.CTkLabel(root, text="Sort by:").place(relx=0.8, rely=0.05)
sort_optionmenu = ctk.CTkOptionMenu(root, variable=sort_var, values=["Date", "Name"], command=lambda _: display_images())
sort_optionmenu.place(relx=0.85, rely=0.05)

content_frame = ctk.CTkScrollableFrame(root)
content_frame.place(relx=0.02, rely=0.12, relwidth=0.96, relheight=0.55)

hl7_textbox = ctk.CTkTextbox(root)
hl7_textbox.place(relx=0.02, rely=0.7, relwidth=0.96, relheight=0.1)

summary_box = ctk.CTkTextbox(root)
summary_box.place(relx=0.02, rely=0.82, relwidth=0.96, relheight=0.1)

root.mainloop()


C:\Users\maryam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkLabel Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
